In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.utils.class_weight import compute_class_weight
import os, gc
from sklearn.metrics import classification_report, roc_auc_score, f1_score

2025-05-19 18:55:58.484488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747680958.682894      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747680958.748389      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [3]:
df = pd.read_csv("/kaggle/input/chexpert-v10-small/CheXpert-v1.0-small/train.csv")
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [4]:
df = df.fillna(0)
df_frontal = df[df['Frontal/Lateral'] == 'Frontal']
df_frontal = df_frontal[~df_frontal.isin([-1]).any(axis=1)]
#df_frontal= df_frontal.drop(["Sex","Age","AP/PA","Frontal/Lateral","Enlarged Cardiomediastinum","Lung Opacity","Lung Lesion","Edema","Consolidation","Pleural Effusion","Pleural Other","Fracture","Support Devices"],axis=1)
df_frontal= df_frontal.drop(["Sex","Age","AP/PA","Frontal/Lateral"],axis=1)
df_frontal['Path'] = df_frontal['Path'].apply(
    lambda x: f"/kaggle/input/chexpert-v10-small/{x}" if isinstance(x, str) else x
)

In [5]:
df_frontal.head()

,Path,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
df_frontal.shape

(118286, 15)

In [7]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 96
EPOCHS = 15
LABEL_COLUMNS = df_frontal.columns[1:] # List your 14 label columns here
base_models = {
    "VGG16": VGG16,
    "VGG19": VGG19,
    "ResNet50": ResNet50,
    "DenseNet121": DenseNet121
}

In [8]:
df_frontal = df_frontal[~(df_frontal[LABEL_COLUMNS] == 0).all(axis=1)]

In [9]:
df_frontal.shape

(115526, 15)

In [10]:
df_frontal.head()

,Path,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,/kaggle/input/chexpert-v10-small/CheXpert-v1.0...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# Step 1: Get total number of training samples
n_samples = len(df_frontal)

# Step 2: Count how many images have each disease
label_counts = df_frontal[LABEL_COLUMNS].sum()

label_counts

No Finding                    16895.0
Enlarged Cardiomediastinum     6046.0
Cardiomegaly                  15442.0
Lung Opacity                  48384.0
Lung Lesion                    4397.0
Edema                         34105.0
Consolidation                  7746.0
Pneumonia                      2974.0
Atelectasis                   21317.0
Pneumothorax                  13702.0
Pleural Effusion              50125.0
Pleural Other                  1427.0
Fracture                       4989.0
Support Devices               68400.0
dtype: float64

In [12]:
X = df_frontal['Path'].values
y = df_frontal[LABEL_COLUMNS].values

In [13]:
X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.1, random_state=42)


In [15]:
X_train.shape, X_test.shape, X_val.shape

((93575,), (10398,), (11553,))

In [16]:
train_df = pd.DataFrame(X_train, columns=["Path"])
train_labels = pd.DataFrame(y_train, columns=LABEL_COLUMNS)
train_df = pd.concat([train_df, train_labels], axis=1)

val_df = pd.DataFrame(X_val, columns=["Path"])
val_labels = pd.DataFrame(y_val, columns=LABEL_COLUMNS)
val_df = pd.concat([val_df, val_labels], axis=1)

test_df = pd.DataFrame(X_test, columns=["Path"])
test_labels = pd.DataFrame(y_test, columns=LABEL_COLUMNS)
test_df = pd.concat([test_df, test_labels], axis=1)

In [17]:
def create_bootstrap_sample(df, seed=None):
    return df.sample(frac=1.0, replace=True, random_state=seed).reset_index(drop=True)

In [18]:
def compute_label_weights(df, label_cols):
    weights = {}
    for col in label_cols:
        classes = df[col].values
        class_weights = compute_class_weight(class_weight='balanced', classes=[0, 1], y=classes)
        weights[col] = class_weights[1]
    return weights

In [19]:
class WeightedImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, x_col, y_cols, batch_size, img_size,
                 datagen, label_weights, shuffle=True):
        self.df = dataframe.copy()
        self.x_col = x_col
        self.y_cols = y_cols
        self.batch_size = batch_size
        self.img_size = img_size
        self.datagen = datagen
        self.label_weights = label_weights
        self.shuffle = shuffle
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = np.zeros((len(batch_df), *self.img_size, 3))
        y = np.zeros((len(batch_df), len(self.y_cols)))
        sample_weights = np.zeros(len(batch_df))

        for i, (_, row) in enumerate(batch_df.iterrows()):
            img = tf.keras.preprocessing.image.load_img(row[self.x_col], target_size=self.img_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            X[i] = self.datagen.random_transform(img_array / 255.).astype('float16')
            y[i] = row[self.y_cols].values.astype('float32')
            sample_weights[i] = sum(y[i][j] * self.label_weights[self.y_cols[j]] for j in range(len(self.y_cols)))

        return X.astype('float16'), y, sample_weights

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [20]:
def build_model(base_model_fn, weights, input_shape=(320, 320, 3), num_classes=14):
    base_model = base_model_fn(include_top=False, weights=weights, input_shape=input_shape)
    for layer in base_model.layers[-10:]: #unfreeze the last 10 layers
        layer.trainable = True
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    return tf.keras.Model(inputs, outputs)

In [21]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision = tf.keras.metrics.Precision(thresholds=threshold)
        self.recall = tf.keras.metrics.Recall(thresholds=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()


In [22]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
metrics = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    F1Score(name='f1_score')
]

I0000 00:00:1747680973.757494      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [23]:
val_gen = WeightedImageDataGenerator(
    dataframe=val_df,
    x_col='Path',
    y_cols=LABEL_COLUMNS,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    datagen=datagen,
    label_weights=compute_label_weights(val_df, LABEL_COLUMNS),
    shuffle=False
)

In [24]:
early_stopping_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    for _ in range(4)
]

In [25]:
# class NaNLossStopper(tf.keras.callbacks.Callback):
#     def on_train_batch_end(self, batch, logs=None):
#         loss = logs.get('loss')
#         if loss is not None and (np.isnan(loss) or np.isinf(loss)):
#             print(f"🚨 NaN loss detected at batch {batch}. Stopping training.")
#             self.model.stop_training = True

In [26]:
trained_models = {}

In [27]:
tf.keras.backend.clear_session()
gc.collect()
tf.config.experimental.reset_memory_stats('GPU:0')

In [28]:
print(f"\nTraining DenseNet121 on bootstrap sample...")

df_sample = create_bootstrap_sample(train_df, seed=4)
label_weights = compute_label_weights(df_sample, LABEL_COLUMNS)

print(f"\nLabel Weights during DenseNet121 training : \n{label_weights}")
    
train_gen = WeightedImageDataGenerator(
    dataframe=df_sample,
    x_col='Path',
    y_cols=LABEL_COLUMNS,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    datagen=datagen,
    label_weights=label_weights
)

        
model = build_model(base_models["DenseNet121"], "/kaggle/input/weights/tensorflow2/default/1/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5" ,input_shape=(*IMG_SIZE, 3), num_classes=len(LABEL_COLUMNS))
model.compile(
    optimizer=Adam(
    learning_rate=1e-4, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-07, 
    amsgrad=False,
    name='Adam',
    clipnorm=1.0
    ),
    loss='binary_crossentropy',
    metrics=metrics
)
        
model.fit(
    train_gen,
    epochs=10,
    verbose=1,
    validation_data=val_gen,
    callbacks=[early_stopping_callbacks[3]]
)
model_path = f"model_DenseNet121.h5"
model.save(model_path)
print(f"Model DenseNet121 saved to {model_path}")
trained_models["DenseNet121"] = model
tf.keras.backend.clear_session()
gc.collect()
tf.config.experimental.reset_memory_stats('GPU:0')


Training DenseNet121 on bootstrap sample...

Label Weights during DenseNet121 training : 
{'No Finding': 3.446846913216443, 'Enlarged Cardiomediastinum': 9.688858977013874, 'Cardiomegaly': 3.7403069789751378, 'Lung Opacity': 1.1940765127733965, 'Lung Lesion': 12.899779432037496, 'Edema': 1.6881652534728486, 'Consolidation': 7.44905269861487, 'Pneumonia': 19.333677685950413, 'Atelectasis': 2.681846841682907, 'Pneumothorax': 4.20297340998922, 'Pleural Effusion': 1.152798994727246, 'Pleural Other': 42.341628959276015, 'Fracture': 11.244292237442922, 'Support Devices': 0.843261120322976}


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1747681164.972598      92 service.cc:148] XLA service 0x7bc558006110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747681164.973380      92 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747681178.853537      92 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747681286.060622      92 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


975/975 ━━━━━━━━━━━━━━━━━━━━ 2400s 2s/step - accuracy: 0.8354 - auc: 0.8033 - f1_score: 0.4821 - loss: 3.1358 - precision: 0.5778 - recall: 0.4149 - val_accuracy: 0.8664 - val_auc: 0.8701 - val_f1_score: 0.5808 - val_loss: 2.7380 - val_precision: 0.6825 - val_recall: 0.5055
Epoch 2/10
975/975 ━━━━━━━━━━━━━━━━━━━━ 800s 821ms/step - accuracy: 0.8684 - auc: 0.8728 - f1_score: 0.5985 - loss: 2.6182 - precision: 0.6791 - recall: 0.5350 - val_accuracy: 0.8692 - val_auc: 0.8818 - val_f1_score: 0.6265 - val_loss: 2.6759 - val_precision: 0.6565 - val_recall: 0.5991
Epoch 3/10
975/975 ━━━━━━━━━━━━━━━━━━━━ 780s 799ms/step - accuracy: 0.8756 - auc: 0.8912 - f1_score: 0.6299 - loss: 2.3839 - precision: 0.6909 - recall: 0.5789 - val_accuracy: 0.8623 - val_auc: 0.8760 - val_f1_score: 0.6194 - val_loss: 2.7988 - val_precision: 0.6274 - val_recall: 0.6116
Epoch 4/10
975/975 ━━━━━━━━━━━━━━━━━━━━ 766s 785ms/step - accuracy: 0.8845 - auc: 0.9099 - f1_score: 0.6653 - loss: 2.0899 - precision: 0.7080 - reca

In [29]:
print(f"\nTraining VGG19 on bootstrap sample...")

df_sample = create_bootstrap_sample(train_df, seed=2)
label_weights = compute_label_weights(df_sample, LABEL_COLUMNS)

print(f"\nLabel Weights during VGG19 training : \n{label_weights}")
    
train_gen = WeightedImageDataGenerator(
    dataframe=df_sample,
    x_col='Path',
    y_cols=LABEL_COLUMNS,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    datagen=datagen,
    label_weights=label_weights
)

        
model = build_model(base_models["VGG19"],"/kaggle/input/weights/tensorflow2/default/1/vgg19_weights_tf_dim_ordering_tf_kernels_notop (1).h5", input_shape=(*IMG_SIZE, 3), num_classes=len(LABEL_COLUMNS))
model.compile(
    optimizer=Adam(
    learning_rate=1e-4, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-07, 
    amsgrad=False,
    name='Adam',
    clipnorm=1.0
    ),
    loss='binary_crossentropy',
    metrics=metrics
)
        
model.fit(
    train_gen,
    epochs=EPOCHS,
    verbose=1,
    validation_data=val_gen,
    callbacks=[early_stopping_callbacks[1]]
)
model_path = f"model_VGG19.h5"
model.save(model_path)
print(f"Model VGG19 saved to {model_path}")
trained_models["VGG19"] = model
tf.keras.backend.clear_session()
gc.collect()
tf.config.experimental.reset_memory_stats('GPU:0')


Training VGG19 on bootstrap sample...

Label Weights during VGG19 training : 
{'No Finding': 3.4374770406289032, 'Enlarged Cardiomediastinum': 9.446295174641632, 'Cardiomegaly': 3.797994967123955, 'Lung Opacity': 1.1996794871794871, 'Lung Lesion': 13.10941440179322, 'Edema': 1.6913386111412356, 'Consolidation': 7.707990115321252, 'Pneumonia': 20.440148536478812, 'Atelectasis': 2.7330743618202, 'Pneumothorax': 4.206374179627798, 'Pleural Effusion': 1.146808667091524, 'Pleural Other': 42.49545867393279, 'Fracture': 11.257820019249278, 'Support Devices': 0.8427446954140999}


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1747664676.386043      96 service.cc:148] XLA service 0x7ba7bc002fe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747664676.387110      96 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747664677.938728      96 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-19 14:25:11.671759: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng2{k2=3,k3=0} for conv (f32[96,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[96,64,224,224]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-05-19 14:25:11.743521: E ex

975/975 ━━━━━━━━━━━━━━━━━━━━ 1969s 2s/step - accuracy: 0.8318 - auc: 0.7985 - f1_score: 0.4663 - loss: 3.1654 - precision: 0.5649 - recall: 0.3997 - val_accuracy: 0.8557 - val_auc: 0.8574 - val_f1_score: 0.5783 - val_loss: 2.8808 - val_precision: 0.6224 - val_recall: 0.5400
Epoch 2/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 845s 866ms/step - accuracy: 0.8571 - auc: 0.8472 - f1_score: 0.5491 - loss: 2.8637 - precision: 0.6493 - recall: 0.4757 - val_accuracy: 0.8639 - val_auc: 0.8699 - val_f1_score: 0.5921 - val_loss: 2.7342 - val_precision: 0.6563 - val_recall: 0.5393
Epoch 3/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 803s 823ms/step - accuracy: 0.8641 - auc: 0.8628 - f1_score: 0.5784 - loss: 2.7419 - precision: 0.6704 - recall: 0.5086 - val_accuracy: 0.8712 - val_auc: 0.8696 - val_f1_score: 0.6068 - val_loss: 2.6660 - val_precision: 0.6885 - val_recall: 0.5425
Epoch 4/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 797s 817ms/step - accuracy: 0.8702 - auc: 0.8733 - f1_score: 0.6022 - loss: 2.6034 - precision: 0.6841 - reca

In [27]:
print(f"\nTraining VGG16 on bootstrap sample...")

df_sample = create_bootstrap_sample(train_df, seed=1)
label_weights = compute_label_weights(df_sample, LABEL_COLUMNS)

print(f"\nLabel Weights during VGG16 training : \n{label_weights}")
    
train_gen = WeightedImageDataGenerator(
    dataframe=df_sample,
    x_col='Path',
    y_cols=LABEL_COLUMNS,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    datagen=datagen,
    label_weights=label_weights
)

        
model = build_model(base_models["VGG16"],"/kaggle/input/weights/tensorflow2/default/1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", input_shape=(*IMG_SIZE, 3), num_classes=len(LABEL_COLUMNS))
model.compile(
    optimizer=Adam(
    learning_rate=1e-4, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-07, 
    amsgrad=False,
    name='Adam',
    clipnorm=1.0
    ),
    loss='binary_crossentropy',
    metrics=metrics
)
        
model.fit(
    train_gen,
    epochs=EPOCHS,
    verbose=1,
    validation_data=val_gen,
    callbacks=[early_stopping_callbacks[0]]
)
model_path = f"model_VGG16.h5"
model.save(model_path)
print(f"Model VGG16 saved to {model_path}")
trained_models["VGG16"] = model
tf.keras.backend.clear_session()
gc.collect()
tf.config.experimental.reset_memory_stats('GPU:0')


Training VGG16 on bootstrap sample...

Label Weights during VGG16 training : 
{'No Finding': 3.448625340900715, 'Enlarged Cardiomediastinum': 9.45966437525273, 'Cardiomegaly': 3.737020766773163, 'Lung Opacity': 1.188767213781188, 'Lung Lesion': 12.974902939545203, 'Edema': 1.6703259433793867, 'Consolidation': 7.431305590851334, 'Pneumonia': 19.034784377542717, 'Atelectasis': 2.709648462384896, 'Pneumothorax': 4.198824374046486, 'Pleural Effusion': 1.1499373264187578, 'Pleural Other': 40.82678883071553, 'Fracture': 11.15315852205006, 'Support Devices': 0.8434131304755381}


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1747643020.066979      96 service.cc:148] XLA service 0x7e97b40639c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747643020.067782      96 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747643021.508095      96 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-19 08:24:50.773745: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng19{} for conv (f16[512,512,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f16[96,512,28,28]{3,2,1,0}, f16[96,512,28,28]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-05-19 08:24:50.841457: E external/

975/975 ━━━━━━━━━━━━━━━━━━━━ 1872s 2s/step - accuracy: 0.8336 - auc: 0.7992 - f1_score: 0.4650 - loss: 3.1144 - precision: 0.5732 - recall: 0.3923 - val_accuracy: 0.8608 - val_auc: 0.8607 - val_f1_score: 0.5872 - val_loss: 2.8036 - val_precision: 0.6424 - val_recall: 0.5408
Epoch 2/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 806s 826ms/step - accuracy: 0.8588 - auc: 0.8507 - f1_score: 0.5575 - loss: 2.8137 - precision: 0.6559 - recall: 0.4847 - val_accuracy: 0.8675 - val_auc: 0.8711 - val_f1_score: 0.6059 - val_loss: 2.7067 - val_precision: 0.6656 - val_recall: 0.5561
Epoch 3/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 767s 786ms/step - accuracy: 0.8663 - auc: 0.8673 - f1_score: 0.5872 - loss: 2.6794 - precision: 0.6781 - recall: 0.5179 - val_accuracy: 0.8729 - val_auc: 0.8813 - val_f1_score: 0.6187 - val_loss: 2.6383 - val_precision: 0.6863 - val_recall: 0.5632
Epoch 4/15
975/975 ━━━━━━━━━━━━━━━━━━━━ 772s 792ms/step - accuracy: 0.8722 - auc: 0.8802 - f1_score: 0.6124 - loss: 2.5438 - precision: 0.6919 - reca

In [ ]:
print(f"\nTraining ResNet50 on bootstrap sample...")

df_sample = create_bootstrap_sample(train_df, seed=3)
label_weights = compute_label_weights(df_sample, LABEL_COLUMNS)

print(f"\nLabel Weights during ResNet50 training : \n{label_weights}")
    
train_gen = WeightedImageDataGenerator(
    dataframe=df_sample,
    x_col='Path',
    y_cols=LABEL_COLUMNS,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    datagen=datagen,
    label_weights=label_weights
)

        
model = build_model(base_models["ResNet50"], "/kaggle/input/weights/tensorflow2/default/1/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5" ,input_shape=(*IMG_SIZE, 3), num_classes=len(LABEL_COLUMNS))
model.compile(
    optimizer=Adam(
    learning_rate=1e-4, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-07, 
    amsgrad=False,
    name='Adam',
    clipnorm=1.0
    ),
    loss='binary_crossentropy',
    metrics=metrics
)
        
model.fit(
    train_gen,
    epochs=10,
    verbose=1,
    validation_data=val_gen,
    callbacks=[early_stopping_callbacks[2]]
)
model_path = f"model_ResNet50.h5"
model.save(model_path)
print(f"Model ResNet50 saved to {model_path}")
trained_models["ResNet50"] = model
tf.keras.backend.clear_session()
gc.collect()
tf.config.experimental.reset_memory_stats('GPU:0')

In [ ]:

# for i, (name, model_fn) in enumerate(base_models.items()):
#     print(f"\nTraining {name} on bootstrap sample...")

#     df_sample = create_bootstrap_sample(train_df, seed=i)
#     label_weights = compute_label_weights(df_sample, LABEL_COLUMNS)

#     print(f"\nLabel Weights during {name} training : \n{label_weights}")
    
#     train_gen = WeightedImageDataGenerator(
#         dataframe=df_sample,
#         x_col='Path',
#         y_cols=LABEL_COLUMNS,
#         batch_size=BATCH_SIZE,
#         img_size=IMG_SIZE,
#         datagen=datagen,
#         label_weights=label_weights
#     )

        
#     model = build_model(model_fn, input_shape=(*IMG_SIZE, 3), num_classes=len(LABEL_COLUMNS))
#     model.compile(
#         optimizer=Adam(
#         learning_rate=1e-4, 
#         beta_1=0.9, 
#         beta_2=0.999, 
#         epsilon=1e-07, 
#         amsgrad=False,
#         name='Adam',
#         clipnorm=1.0
#         ),
#         loss='binary_crossentropy',
#         metrics=metrics
#     )
        
#     model.fit(
#         train_gen,
#         epochs=EPOCHS,
#         verbose=1,
#         validation_data=val_gen,
#         callbacks=[early_stopping, NaNLossStopper()]
#     )
#     trained_models[name] = model
#     tf.keras.backend.clear_session()
#     gc.collect()
#     tf.config.experimental.reset_memory_stats('GPU:0')

In [ ]:
def load_images(df, target_size):
    X = []
    for path in df['Path']:
        img = tf.keras.preprocessing.image.load_img(path, target_size=target_size)
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.
        X.append(img)
    return np.array(X)

In [ ]:
X_test = load_images(test_df, IMG_SIZE)
y_test = test_df[LABEL_COLUMNS].values

# Ensemble prediction (average)
preds = [model.predict(X_test) for model in trained_models.values()]
ensemble_preds = np.mean(preds, axis=0)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [ ]:
binary_preds = (ensemble_preds > 0.5).astype(int)
print("\nClassification Report:")
print(classification_report(y_test, binary_preds, target_names=LABEL_COLUMNS))
print("Macro AUC:", roc_auc_score(y_test, ensemble_preds, average='macro'))
print("Macro F1:", f1_score(y_test, binary_preds, average='macro'))
print("Macro Accuracy:", accuracy_score(y_test, binary_preds))
print("Macro Precision:", precision_score(y_test, binary_preds, average='macro'))
print("Macro Recall:", recall_score(y_test, binary_preds, average='macro'))

In [ ]:
print(trained_models)